In [1]:
###显微镜设备自检测试
import time
Version = 'V2' ###V1
if Version == 'V1':
    from Drives.microcontroller_V1 import Microcontroller
    import control.core_V1 as core
    from Drives._def_V1 import *
elif Version == 'V2':
    from Drives.microcontroller_V2 import Microcontroller
    import control.core_V2 as core
    from Drives._def_V2 import *
# 初始化显微镜控制器
microcontroller = Microcontroller(COM='COM1')
# 重置显微镜控制器
microcontroller.reset()
# 初始化马达驱动
microcontroller.initialize_drivers()
# 配置执行器
microcontroller.configure_actuators()
print(显微镜初始化成功)

In [2]:
# 初始化导航控制器
navigationController = core.NavigationController(microcontroller)
# 将Z轴归零
navigationController.home_z()
# 等待Z轴归零完成
t0 = time.time()
while microcontroller.is_busy():
    time.sleep(0.005)
    if time.time() - t0 > 8:
        print('Z轴归零超时')
        break

In [ ]:
# 将X轴归零
t0 = time.time()
navigationController.home_x()
# 等待X轴归零完成
while microcontroller.is_busy():
    time.sleep(0.005)
    if time.time() - t0 > 8:
        print('X轴归零超时')
        break

In [3]:
# 将Y轴归零
t0 = time.time()
navigationController.home_y()
# 等待Y轴归零完成
while microcontroller.is_busy():
    time.sleep(0.005)
    if time.time() - t0 > 8:
        print('Y轴归零超时')
        break

In [ ]:
### 配置PID设置
if Version == 'V1':
    # 根据配置信息，条件性地配置编码器
    if HAS_ENCODER_X == True:
        navigationController.configure_encoder(0, (SCREW_PITCH_X_MM * 1000) / ENCODER_RESOLUTION_UM_X,
                                                            ENCODER_FLIP_DIR_X)
    if HAS_ENCODER_Y == True:
        navigationController.configure_encoder(1, (SCREW_PITCH_Y_MM * 1000) / ENCODER_RESOLUTION_UM_Y,
                                                            ENCODER_FLIP_DIR_Y)
    if HAS_ENCODER_Z == True:
        navigationController.configure_encoder(2, (SCREW_PITCH_Z_MM * 1000) / ENCODER_RESOLUTION_UM_Z,
                                                            ENCODER_FLIP_DIR_Z)
    # 启用PID控制
    navigationController.set_pid_control_enable(0, ENABLE_PID_X)
    navigationController.set_pid_control_enable(1, ENABLE_PID_Y)
    navigationController.set_pid_control_enable(2, ENABLE_PID_Z)
    time.sleep(0.5)
elif Version == 'V2':
    # 根据配置信息，条件性地配置编码器
    if HAS_ENCODER_X == True:
        navigationController.set_axis_PID_arguments(0, PID_P_X, PID_I_X, PID_D_X)
        navigationController.configure_encoder(0, (SCREW_PITCH_X_MM * 1000) / ENCODER_RESOLUTION_UM_X,
                                                            ENCODER_FLIP_DIR_X)
        navigationController.set_pid_control_enable(0, ENABLE_PID_X)
        if HAS_ENCODER_Y == True:
            navigationController.set_axis_PID_arguments(1, PID_P_Y, PID_I_Y, PID_D_Y)
            navigationController.configure_encoder(1, (SCREW_PITCH_Y_MM * 1000) / ENCODER_RESOLUTION_UM_Y,
                                                            ENCODER_FLIP_DIR_Y)
            navigationController.set_pid_control_enable(1, ENABLE_PID_Y)
        if HAS_ENCODER_Z == True:
            navigationController.set_axis_PID_arguments(2, PID_P_Z, PID_I_Z, PID_D_Z)
            navigationController.configure_encoder(2, (SCREW_PITCH_Z_MM * 1000) / ENCODER_RESOLUTION_UM_Z,
                                                            ENCODER_FLIP_DIR_Z)
            navigationController.set_pid_control_enable(2, ENABLE_PID_Z)
    time.sleep(0.5)

In [ ]:
# 设置软件限位
# set software limit
navigationController.set_x_limit_pos_mm(SOFTWARE_POS_LIMIT.X_POSITIVE)
navigationController.set_x_limit_neg_mm(SOFTWARE_POS_LIMIT.X_NEGATIVE)
navigationController.set_y_limit_pos_mm(SOFTWARE_POS_LIMIT.Y_POSITIVE)
navigationController.set_y_limit_neg_mm(SOFTWARE_POS_LIMIT.Y_NEGATIVE)
navigationController.set_z_limit_pos_mm(SOFTWARE_POS_LIMIT.Z_POSITIVE)

In [ ]:
# 移动检查
navigationController.move_x_to(8)
while microcontroller.is_busy():
    time.sleep(0.005)
navigationController.move_y_to(8)
while microcontroller.is_busy():
    time.sleep(0.005)
navigationController.home_z()
while microcontroller.is_busy():
    time.sleep(0.005)
navigationController.home_xy()
while microcontroller.is_busy():
    time.sleep(0.005)